In [1]:
import pandas as pd
%matplotlib inline
import numpy as np



import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) #pra rodar no notebook

In [3]:
trace = go.Heatmap(z=[[1, 20, 30, 50, 1], [20, 1, 60, 80, 30], [30, 60, 1, -10, 20]],
                   x=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],
                   y=['Morning', 'Afternoon', 'Evening'])


trace2 = go.Scatter(
    x=[20, 30, 40],
    y=[50, 60, 70],
    xaxis='x2',
    yaxis='y2'
)

data=[trace, trace2]

layout = go.Layout(
#     xaxis=dict(
#         domain=[0, 1]
#     ),
#     xaxis2=dict(
#         domain=[0, 1]
#     ),
    yaxis = dict(domain = [0,0.5] ) ,
    yaxis2 = dict(domain = [0.6,1], anchor='x2')
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='labelled-heatmap.html')






In [4]:
trace = go.Heatmap(z=[[1, 20, 30],
                      [20, 1, 60],
                      [30, 60, 1]])
data=[trace]
iplot(data, filename='basic-heatmap')

## Testes

In [10]:
def read_gmt_hier(file_name):
    '''
    file_name string -> dataframe and dictionary
    
    Reads .gmt file and returns a Pandas DataFrame and a dictionary with the information of the gmt file in a format ready to be used.
    
    '''
    gmt_names = []
    gmt_desc  = []
    gmt_genes = []
    res =pd.DataFrame()

    with open(file_name, 'r') as f:
        # separar cada elemento separado por tab e guardar eles
        for line in f:
            gmt_names.append(line.split("\t")[0])
            gmt_desc.append(line.split("\t")[1])
            gmt_genes.append(line.split("\t")[2:])

    for i in range(len(gmt_genes)) :

        # apagar \n presente no ultimo gene de cada lista (artefato da leitura do arquivo)
        gmt_genes[i][-1] = gmt_genes[i][-1].replace("\n", "")

        # Poem na forma de um dataframe, cada linha um gene e suas informações relativas 
        temp= pd.DataFrame({'term': [gmt_names[i]]*len(gmt_genes[i]), 'hier':  [gmt_desc[i]]*len(gmt_genes[i]), 'gene' : gmt_genes[i] })
        res = pd.concat([res,temp])

    # reseta o indice
    res = res.reset_index(drop=True)    

    #relação entre nomes e descricões  (é pra isso q essa variavel serve? no original é um datafra esquisito, achei q assim ia ser mais otimizado)   
    path_desc = dict(zip(gmt_names,gmt_desc))

    return res, path_desc , gmt_names

In [11]:
    term2gene , path_desc , merge_p  = read_gmt_hier("data/Reactome_2016_15and100Genes.gmt") #utils.read_gmt_hier(gmt_file)    
   
    ############################################
    f = lambda df: [gene for gene in df["gene"] ]
    dict_genes_por_via = dict(term2gene.groupby(["term"]).apply(f) )
    ################################################

In [20]:
def create_interactive_plot(df, dict_genes_por_via , direction, analysis_name,cut_path):

    heatmap = create_heatmap(df)
    barplot = create_barplot(dict_genes_por_via, df)
    line_plot = plot_number_of_enriched_pathways(cut_path)
    
    data=[heatmap, barplot, line_plot]
    layout = go.Layout( xaxis = dict(domain = [0,0.80] , showticklabels = False), 
                       xaxis2 = dict(domain = [0,0.80] , showticklabels = False), 
                       xaxis3 = dict(domain = [0.85,1], anchor = "y3"), 
                       yaxis  = dict(domain = [0.30,1]) ,  
                       yaxis2 = dict(domain = [0,0.25] ,autorange = 'reversed', anchor = "x2") ,
                       yaxis3 = dict(domain = [0.30,1], anchor = "x3" ))
    figure = go.Figure(data=data, layout=layout)

    plot(figure, filename = analysis_name + "_" +  direction + '.html')

In [21]:
def create_heatmap(df):
    NGs = df.columns.tolist()
    pathways = df.index.tolist()
    valores = [ df[column].tolist() for column in df]

    trace = go.Heatmap(z=valores, y=NGs, x=pathways)
    return trace


def create_barplot(dict_genes_por_via, df):
    barplot = go.Bar(x = df.index.tolist(),
                     y = [ len(dict_genes_por_via[via]) for via in df.index.tolist() ] ,
                     orientation = "v",
                     xaxis = "x2",
                     yaxis = "y2"
                     )
    return barplot

def plot_number_of_enriched_pathways(cut_path):
    plot = go.Scatter(x = cut_path["times_significant_up"].tolist(),
                      y = cut_path.index.tolist(),
                      orientation = "h",
                      mode = 'lines+markers',
                      name = 'lines+markers',
                      xaxis = "x3",
                      yaxis = "y3")
    return plot

In [22]:
create_interactive_plot(df, dict_genes_por_via , "direction", "analise", cut_path)

In [23]:
df = pd.read_csv("Comparando_outputs/outputs_py/path_py_up.csv", index_col=0)


In [4]:
cut_path = pd.read_csv("Comparando_outputs/outputs_py/cut_path_py_up.csv", index_col=0)


In [7]:
plot([plot_number_of_enriched_pathways(cut_path)], "testeerver.html")




'temp-plot.html'

# TA DANDO PROBLEMA NO ANCHOR, FALTA SÓ JUNTAR ESSES 3 PLOTS